# 1. Imports and File selection 

In [1]:
import io
import ipywidgets as widgets
import math
import numpy
import psycopg
import pandas as pd
import requests
import sqlite3
import sys
import tqdm
import warnings

from config import load_config
from ipyfilechooser import FileChooser
from scipy import stats
from scipy.stats import ttest_ind
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import insert
from sqlite3 import Error
from sqlite3 import IntegrityError

## Select Baseline .csv File

In [2]:
starting_directory = '/Users'
baseline_chooser = FileChooser(starting_directory)
display(baseline_chooser)

FileChooser(path='/Users', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Change…

## Select Tap .csv File

In [3]:
tap_chooser=FileChooser('/Users')
display(tap_chooser)

FileChooser(path='/Users', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Change…

## Select Post Stimulus Arousal .csv File

In [4]:
psa_chooser = FileChooser('/Users')
display(psa_chooser)

FileChooser(path='/Users', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Change…

In [5]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 
           'Neuron_Genes_Screen', 'PD_GWAS_Locus71_Screen', 'ASD_WGS_Screen']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

Select(description='Screen:', options=('PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'N…

In [6]:
Screen=screen_chooser.value
folder_path=baseline_chooser.selected_path
print(folder_path)

/Users/gurmehak/Documents/RankinLab/Test_Datasets/ASD_WGS_2025


## Read baseline, tap and post stimulus arousal (psa) data

In [7]:
# Read the baseline file
baseline_output = pd.read_csv(baseline_chooser.selected, index_col=0)#.drop(columns=['index'])

print(f"\nShape of the baseline .csv file: {baseline_output.shape}")

# Print the first five rows of the file
baseline_output.head()


Shape of the baseline .csv file: (1083420, 21)


,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,...,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,dataset,Gene,Allele
2236,490.042,28,19,0.0589,0.0647,0.111,0.0921,1.1388,0.130875,3.8,...,46.7,30.2,0.0073,6.729,A1025,20241025,ASD_Screen,N2,N2,N2
2237,490.084,28,19,0.0647,0.0683,0.111,0.0915,1.1366,0.130529,3.2,...,46.9,30.9,0.0094,6.730,A1025,20241025,ASD_Screen,N2,N2,N2
2238,490.125,28,19,0.0672,0.0757,0.111,0.0916,1.1354,0.130261,3.4,...,45.3,31.0,0.0111,6.731,A1025,20241025,ASD_Screen,N2,N2,N2
2239,490.167,28,19,0.0825,0.1098,0.111,0.0916,1.1350,0.129839,4.6,...,45.6,31.0,0.0136,6.732,A1025,20241025,ASD_Screen,N2,N2,N2
2240,490.202,28,19,0.0849,0.1072,0.111,0.0915,1.1389,0.130338,4.4,...,46.3,30.9,0.0122,6.734,A1025,20241025,ASD_Screen,N2,N2,N2


In [8]:
# Read the tap file
tap_output = pd.read_csv(tap_chooser.selected, index_col=0)

print(f"\nShape of the tap .csv file: {tap_output.shape}")

# Print the first five rows of the file
tap_output.head()


Shape of the tap .csv file: (15298, 13)


,time,dura,dist,prob,speed,plate,Date,Plate_id,Screen,taps,dataset,Gene,Allele
0,599.976,3.47,0.826,0.800000,0.238040,1,20250526,A0526,ASD_Screen,1.0,F33H2.3_tm1628,F33H2.3,tm1628
1,609.994,2.21,0.505,0.761905,0.228507,1,20250526,A0526,ASD_Screen,2.0,F33H2.3_tm1628,F33H2.3,tm1628
2,619.990,2.27,0.605,0.782609,0.266520,1,20250526,A0526,ASD_Screen,3.0,F33H2.3_tm1628,F33H2.3,tm1628
3,629.979,2.13,0.519,0.739130,0.243662,1,20250526,A0526,ASD_Screen,4.0,F33H2.3_tm1628,F33H2.3,tm1628
4,639.981,1.97,0.455,0.666667,0.230964,1,20250526,A0526,ASD_Screen,5.0,F33H2.3_tm1628,F33H2.3,tm1628


In [9]:
# Read the psa file
psa_output = pd.read_csv(psa_chooser.selected, index_col=0)

print(f"\nShape of the tap .csv file: {psa_output.shape}")

# Print the first five rows of the file
psa_output.head()


Shape of the tap .csv file: (15677, 24)


,Experiment,Tap_num,Plate_id,Date,Screen,dataset,Gene,Allele,Time,n,...,Tap,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
0,1,1,A1025,20241025,ASD_Screen,N2,N2,N2,607,27.108108,...,0.0,0.092409,1.109945,0.127168,7.777027,0.431595,67.251351,41.963514,0.015874,4.108203
1,1,2,A1025,20241025,ASD_Screen,N2,N2,N2,617,28.013514,...,0.0,0.091112,1.121291,0.127303,12.132432,0.381635,68.912162,39.686486,0.023738,3.705216
2,1,3,A1025,20241025,ASD_Screen,N2,N2,N2,627,29.000000,...,0.0,0.093401,1.130608,0.129129,11.096000,0.320747,53.538667,35.998667,0.024719,2.789467
3,1,4,A1025,20241025,ASD_Screen,N2,N2,N2,637,27.179104,...,0.0,0.093554,1.144525,0.130611,9.980597,0.290284,45.450746,34.279104,0.027582,3.432299
4,1,5,A1025,20241025,ASD_Screen,N2,N2,N2,647,25.148148,...,0.0,0.091465,1.144035,0.131804,7.879630,0.275704,42.940741,35.300000,0.022185,4.504981


# 2. DataFrame preparation

### 2.1. Tap Data

In [10]:
# Dataframe for first tap
PD_first_tap = (
    tap_output[(tap_output.taps==1)]
    .reset_index().drop(columns="index")
    .rename(columns={"dura": "init_dura", "prob": "init_prob", "speed": "init_speed"}, errors="raise")
)

PD_first_tap.head()

,time,init_dura,dist,init_prob,init_speed,plate,Date,Plate_id,Screen,taps,dataset,Gene,Allele
0,599.976,3.47,0.826,0.800000,0.238040,1,20250526,A0526,ASD_Screen,1.0,F33H2.3_tm1628,F33H2.3,tm1628
1,599.980,2.57,0.597,0.857143,0.232296,2,20250526,A0526,ASD_Screen,1.0,F33H2.3_tm1628,F33H2.3,tm1628
2,599.980,3.03,0.634,0.789474,0.209241,3,20250526,B0526,ASD_Screen,1.0,F33H2.3_tm1628,F33H2.3,tm1628
3,599.966,3.34,0.886,1.000000,0.265269,4,20250526,C0526,ASD_Screen,1.0,F33H2.3_tm1628,F33H2.3,tm1628
4,599.977,3.35,0.822,0.833333,0.245373,5,20250526,C0526,ASD_Screen,1.0,F33H2.3_tm1628,F33H2.3,tm1628


In [11]:
# Dataframe for recovery taps
PD_recov_taps = (
    tap_output[(tap_output.taps==31)]
    .reset_index().drop(columns="index")
    .rename(columns={"dura": "recov_dura", "prob": "recov_prob", "speed":"recov_speed"})
)

PD_recov_taps.head()

,time,recov_dura,dist,recov_prob,recov_speed,plate,Date,Plate_id,Screen,taps,dataset,Gene,Allele
0,1189.976,2.04,0.480,0.571429,0.235294,1,20250526,A0526,ASD_Screen,31.0,F33H2.3_tm1628,F33H2.3,tm1628
1,1189.981,3.66,0.763,0.764706,0.208470,2,20250526,A0526,ASD_Screen,31.0,F33H2.3_tm1628,F33H2.3,tm1628
2,1189.989,3.50,0.814,0.818182,0.232571,3,20250526,B0526,ASD_Screen,31.0,F33H2.3_tm1628,F33H2.3,tm1628
3,1189.989,3.91,1.054,1.000000,0.269565,4,20250526,C0526,ASD_Screen,31.0,F33H2.3_tm1628,F33H2.3,tm1628
4,1189.968,3.92,1.018,1.000000,0.259694,5,20250526,C0526,ASD_Screen,31.0,F33H2.3_tm1628,F33H2.3,tm1628


In [12]:
# Dataframe for last three taps
PD_final_taps = (
    tap_output[((tap_output.taps >= 28) & (tap_output.taps <= 30))]
    .groupby(["dataset", "Date","Plate_id","Screen","Gene","Allele","plate"])
    .mean()
    .reset_index()
    .rename(columns={"dura": "final_dura", "prob": "final_prob", "speed": "final_speed"}, errors="raise")
)

PD_final_taps.head()

,dataset,Date,Plate_id,Screen,Gene,Allele,plate,time,final_dura,dist,final_prob,final_speed,taps
0,F33H2.3_tm1628,20250526,A0526,ASD_Screen,F33H2.3,tm1628,1,879.970667,0.693333,0.158333,0.605238,0.228300,29.0
1,F33H2.3_tm1628,20250526,A0526,ASD_Screen,F33H2.3,tm1628,2,879.979667,0.313333,0.057333,0.494426,0.183275,29.0
2,F33H2.3_tm1628,20250526,B0526,ASD_Screen,F33H2.3,tm1628,3,879.972000,0.663333,0.148000,0.518568,0.219684,29.0
3,F33H2.3_tm1628,20250526,C0526,ASD_Screen,F33H2.3,tm1628,4,879.970667,0.816667,0.153000,0.786616,0.185585,29.0
4,F33H2.3_tm1628,20250526,C0526,ASD_Screen,F33H2.3,tm1628,5,879.971667,0.553333,0.108333,0.466667,0.200407,29.0


In [13]:
# Dataframe to analyse habituation behaviour after merging first tap and final taps

PD_habit_levels = pd.merge(
    PD_first_tap, 
    PD_final_taps, 
    on =['dataset', 'plate', "Plate_id", "Screen", "Gene", "Allele", "Date"], how ='left'
).drop(columns=['time_x','time_y','dist_x','dist_y', 'taps_x', 'taps_y']).dropna()

PD_habit_levels['habit_dura'] = PD_habit_levels['init_dura'] - PD_habit_levels['final_dura']

PD_habit_levels['habit_prob'] = PD_habit_levels['init_prob'] - PD_habit_levels['final_prob']

PD_habit_levels['habit_speed'] = PD_habit_levels['init_speed'] - PD_habit_levels['final_speed']

In [14]:
# Continue to analyse habituation behaviour after merging with recovery taps

if PD_recov_taps.empty:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='outer')
else:
    PD_habituation = pd.merge(PD_habit_levels, PD_recov_taps, on =['dataset','plate',"Plate_id","Screen","Gene","Allele","Date"], how ='left')

if Screen not in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    PD_habituation = PD_habituation.dropna() 

PD_habituation['recovery_dura']=(PD_habituation.recov_dura-PD_habituation.init_dura)/PD_habituation.init_dura*100

PD_habituation['recovery_prob']=(PD_habituation.recov_prob-PD_habituation.init_prob)/PD_habituation.init_prob*100

PD_habituation['recovery_speed']=(PD_habituation.recov_speed-PD_habituation.init_speed)/PD_habituation.init_speed*100

PD_habituation['memory_retention_dura']=(PD_habituation.recov_dura-PD_habituation.final_dura)

PD_habituation['memory_retention_prob']=(PD_habituation.recov_prob-PD_habituation.final_prob)

PD_habituation['memory_retention_speed']=(PD_habituation.recov_speed-PD_habituation.final_speed)


# Rename `PD_habituation` to `tap_data` based on the condition below
if Screen in ['Neuron_Genes_Screen', 'G-Proteins_Screen']:
    tap_data=PD_habituation.dropna(subset = ['init_dura', 'init_prob', 'init_speed', 'plate', 'Date', 'Plate_id',
       'Screen', 'dataset', 'Gene', 'Allele', 'final_dura', 'final_prob',
       'final_speed', 'habit_dura', 'habit_prob', 'habit_speed'])
else:
    tap_data=PD_habituation.dropna() 


# Display final dataframe
tap_data.head()


,init_dura,init_prob,init_speed,plate,Date,Plate_id,Screen,dataset,Gene,Allele,...,dist,recov_prob,recov_speed,taps,recovery_dura,recovery_prob,recovery_speed,memory_retention_dura,memory_retention_prob,memory_retention_speed
0,3.47,0.800000,0.238040,1,20250526,A0526,ASD_Screen,F33H2.3_tm1628,F33H2.3,tm1628,...,0.480,0.571429,0.235294,31.0,-41.210375,-28.571429,-1.153682,1.346667,-0.033810,0.006995
1,2.57,0.857143,0.232296,2,20250526,A0526,ASD_Screen,F33H2.3_tm1628,F33H2.3,tm1628,...,0.763,0.764706,0.208470,31.0,42.412451,-10.784314,-10.256657,3.346667,0.270280,0.025195
2,3.03,0.789474,0.209241,3,20250526,B0526,ASD_Screen,F33H2.3_tm1628,F33H2.3,tm1628,...,0.814,0.818182,0.232571,31.0,15.511551,3.636364,11.150068,2.836667,0.299614,0.012888
3,3.34,1.000000,0.265269,4,20250526,C0526,ASD_Screen,F33H2.3_tm1628,F33H2.3,tm1628,...,1.054,1.000000,0.269565,31.0,17.065868,0.000000,1.619393,3.093333,0.213384,0.083980
4,3.35,0.833333,0.245373,5,20250526,C0526,ASD_Screen,F33H2.3_tm1628,F33H2.3,tm1628,...,1.018,1.000000,0.259694,31.0,17.014925,20.000000,5.836313,3.366667,0.533333,0.059287


### 2.2. PSA data

In [15]:
# function to calculate Inidial, Final, Peak, ect values for specified column (metric)

def summary_metrics(df, metric = 'Instantaneous Speed'):

    initial = df[metric].iloc[0]
    recovery = df[metric].iloc[-1]
    peak = df[metric].max()
    mean = df[metric].mean()
    peak_id = df[metric].values.argmax()
    initial_to_peak = df[metric].iloc[: peak_id+1].mean()
    peak_to_recovery = df[metric].iloc[peak_id:].mean()
    

    return pd.Series({
        f'PSA Initial {metric}': initial, 
        f'PSA Recovery {metric}': recovery, 
        f'PSA Peak {metric}': peak,
        f'PSA Initial_to_peak {metric}': initial_to_peak, 
        f'PSA Peak_to_recovery {metric}': peak_to_recovery,
        f'PSA Average {metric}': mean
        })

In [16]:
warnings.filterwarnings('ignore')

# columns to summarize
metrics_to_summarize = ['Instantaneous Speed', 'Bias', 'Angular Speed', 'Aspect Ratio', 'Kink', 'Curve', 'Crab']

# standard columns
group_cols = ['Experiment', 'Plate_id', 'Date', 'Screen', 'dataset', 'Gene', 'Allele']

# pass each column to summarise through `summary_metrics` function and merge the summarised values to psa_output
psa_data = psa_output[group_cols]
for metric in metrics_to_summarize:
    summary = psa_output.groupby(group_cols).apply(lambda x: summary_metrics(x, metric)).reset_index()
    psa_data = pd.merge(psa_data, summary, on=group_cols, how='left')

In [17]:
psa_data.head()

,Experiment,Plate_id,Date,Screen,dataset,Gene,Allele,PSA Initial Instantaneous Speed,PSA Recovery Instantaneous Speed,PSA Peak Instantaneous Speed,...,PSA Peak Curve,PSA Initial_to_peak Curve,PSA Peak_to_recovery Curve,PSA Average Curve,PSA Initial Crab,PSA Recovery Crab,PSA Peak Crab,PSA Initial_to_peak Crab,PSA Peak_to_recovery Crab,PSA Average Crab
0,1,A1025,20241025,ASD_Screen,N2,N2,N2,0.085788,0.134799,0.233648,...,41.963514,41.963514,33.670672,33.670672,0.015874,0.016918,0.027582,0.022978,0.017078,0.0175
1,1,A1025,20241025,ASD_Screen,N2,N2,N2,0.085788,0.134799,0.233648,...,41.963514,41.963514,33.670672,33.670672,0.015874,0.016918,0.027582,0.022978,0.017078,0.0175
2,1,A1025,20241025,ASD_Screen,N2,N2,N2,0.085788,0.134799,0.233648,...,41.963514,41.963514,33.670672,33.670672,0.015874,0.016918,0.027582,0.022978,0.017078,0.0175
3,1,A1025,20241025,ASD_Screen,N2,N2,N2,0.085788,0.134799,0.233648,...,41.963514,41.963514,33.670672,33.670672,0.015874,0.016918,0.027582,0.022978,0.017078,0.0175
4,1,A1025,20241025,ASD_Screen,N2,N2,N2,0.085788,0.134799,0.233648,...,41.963514,41.963514,33.670672,33.670672,0.015874,0.016918,0.027582,0.022978,0.017078,0.0175


# 3. Run Statistics (T-Test and mean sample distance) on Data

## 3.1 Generate dataframes conditioned by `baseline` (True/False) and `allele` (True/False)

In [18]:
def get_output_byplate(output, baseline=["true", "false", "psa"], allele = [False, True]):
    """
    Aggregates data by 'Plate_id','Date','Screen','dataset','Gene','Allele'

    Parameters:
        output (pd.DataFrame): Input DataFrame (either baseline_output or tap_data)
        baseline (boolean): whether data is baseline (True) or tap response (False)
        allele (boolean): group by allele (True) or group by gene (False)

    Returns:
        A DataFrame with plate-level averages
    """
    
    # columns to delete if baseline = true
    if baseline == "true":
        drop_col = ['Plate_id','n','Number','Time','Screen','Date','Allele']
    # columns to delete if baseline = false
    elif baseline == "false":
        drop_col = ['Plate_id','Screen','Date','Allele','dist','plate','time',
                       'taps','recov_dura','recov_prob','recov_speed']
    # columns to delete if baseline = psa
    else: 
        drop_col = ['Experiment', 'Plate_id', 'Date', 'Screen', 'Allele']

    drop_col.append('Gene') if allele else drop_col.append('dataset')
     
    output_byplate = output.groupby(
        by=['Plate_id','Date','Screen','dataset','Gene','Allele'],
        as_index=False).mean().drop(columns=drop_col)
    
    return output_byplate

#### 3.1.1 `baseline` = True, `allele` = False

In [19]:
baseline_output_byplate=get_output_byplate(baseline_output, baseline= "true", allele=False)

print(f"Shape: {baseline_output_byplate.shape}")

baseline_output_byplate.head()

Shape: (269, 13)


,Gene,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
0,N2,0.073678,0.083914,0.219774,0.095801,1.095087,0.128080,3.606908,0.278274,46.150592,30.436690,0.009030,5.335810
1,kvs-4,0.092980,0.088686,0.315750,0.098395,1.117168,0.133686,3.785957,0.270687,45.135063,30.019778,0.009494,6.998394
2,kvs-4,0.055899,0.053505,0.124086,0.095603,1.057266,0.124128,3.296072,0.267519,44.771540,27.879080,0.007291,6.774967
3,kvs-4,0.046281,0.050333,0.099498,0.100498,1.037063,0.127650,2.524305,0.246843,42.119075,26.362506,0.006290,4.690770
4,kvs-5,0.042761,0.046602,0.088628,0.093991,1.041901,0.122121,2.730337,0.257110,43.428565,26.357622,0.006338,5.102652


#### 3.1.2 `baseline` = False, `allele` = False

In [20]:
tap_data_byplate=get_output_byplate(tap_data, baseline="false", allele=False)

print(f"Shape: {tap_data_byplate.shape}")

tap_data_byplate.head()

Shape: (265, 16)


,Gene,init_dura,init_prob,init_speed,final_dura,final_prob,final_speed,habit_dura,habit_prob,habit_speed,recovery_dura,recovery_prob,recovery_speed,memory_retention_dura,memory_retention_prob,memory_retention_speed
0,N2,2.820,0.445635,0.253934,0.970000,0.252952,0.224616,1.850000,0.192683,0.029318,-32.804794,286.535304,1.213328,0.850000,0.461334,0.032222
1,kvs-4,3.570,0.823276,0.278881,0.773333,0.379499,0.220342,2.796667,0.443777,0.058539,-27.384549,20.693109,6.288016,1.806667,0.604876,0.074058
2,kvs-4,3.455,0.949074,0.231383,0.768333,0.731218,0.212478,2.686667,0.217856,0.018906,11.550803,2.571429,-5.294891,3.086667,0.241004,0.006614
3,kvs-4,3.575,0.926587,0.229941,0.901667,0.781304,0.241574,2.673333,0.145283,-0.011633,-19.422710,-0.573257,4.987005,1.978333,0.139284,-0.000010
4,kvs-5,3.000,0.952240,0.240154,0.765000,0.740575,0.214348,2.235000,0.211665,0.025807,1.674547,3.379360,0.955550,2.325000,0.243800,0.028717


#### 3.1.3 `baseline` = True, `allele` = True

In [21]:
baseline_output_allele_byplate = get_output_byplate(baseline_output,baseline="true", allele=True)

print(f"Shape: {baseline_output_allele_byplate.shape}")

baseline_output_allele_byplate.head()

Shape: (269, 13)


,dataset,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
0,N2,0.073678,0.083914,0.219774,0.095801,1.095087,0.128080,3.606908,0.278274,46.150592,30.436690,0.009030,5.335810
1,kvs-4_sy1622,0.092980,0.088686,0.315750,0.098395,1.117168,0.133686,3.785957,0.270687,45.135063,30.019778,0.009494,6.998394
2,kvs-4_tm10514,0.055899,0.053505,0.124086,0.095603,1.057266,0.124128,3.296072,0.267519,44.771540,27.879080,0.007291,6.774967
3,kvs-4_tm14987,0.046281,0.050333,0.099498,0.100498,1.037063,0.127650,2.524305,0.246843,42.119075,26.362506,0.006290,4.690770
4,kvs-5_tm6152,0.042761,0.046602,0.088628,0.093991,1.041901,0.122121,2.730337,0.257110,43.428565,26.357622,0.006338,5.102652


#### 3.1.4 `baseline` = False, `allele` = True

In [22]:
tap_data_allele_byplate = get_output_byplate(tap_data, baseline="false", allele=True)

print(f"Shape: {tap_data_allele_byplate.shape}")

tap_data_allele_byplate.head()

Shape: (265, 16)


,dataset,init_dura,init_prob,init_speed,final_dura,final_prob,final_speed,habit_dura,habit_prob,habit_speed,recovery_dura,recovery_prob,recovery_speed,memory_retention_dura,memory_retention_prob,memory_retention_speed
0,N2,2.820,0.445635,0.253934,0.970000,0.252952,0.224616,1.850000,0.192683,0.029318,-32.804794,286.535304,1.213328,0.850000,0.461334,0.032222
1,kvs-4_sy1622,3.570,0.823276,0.278881,0.773333,0.379499,0.220342,2.796667,0.443777,0.058539,-27.384549,20.693109,6.288016,1.806667,0.604876,0.074058
2,kvs-4_tm10514,3.455,0.949074,0.231383,0.768333,0.731218,0.212478,2.686667,0.217856,0.018906,11.550803,2.571429,-5.294891,3.086667,0.241004,0.006614
3,kvs-4_tm14987,3.575,0.926587,0.229941,0.901667,0.781304,0.241574,2.673333,0.145283,-0.011633,-19.422710,-0.573257,4.987005,1.978333,0.139284,-0.000010
4,kvs-5_tm6152,3.000,0.952240,0.240154,0.765000,0.740575,0.214348,2.235000,0.211665,0.025807,1.674547,3.379360,0.955550,2.325000,0.243800,0.028717


In [23]:
# tap_data_allele_byplate[tap_data_allele_byplate.dataset=='N2_XJ1']

#### 3.1.5 `baseline` = "psa" , `allele` = False

In [24]:
psa_data_byplate = get_output_byplate(psa_data, baseline="psa", allele=False)

print(f"Shape: {psa_data_byplate.shape}")

psa_data_byplate.head()

Shape: (269, 43)


,Gene,PSA Initial Instantaneous Speed,PSA Recovery Instantaneous Speed,PSA Peak Instantaneous Speed,PSA Initial_to_peak Instantaneous Speed,PSA Peak_to_recovery Instantaneous Speed,PSA Average Instantaneous Speed,PSA Initial Bias,PSA Recovery Bias,PSA Peak Bias,...,PSA Peak Curve,PSA Initial_to_peak Curve,PSA Peak_to_recovery Curve,PSA Average Curve,PSA Initial Crab,PSA Recovery Crab,PSA Peak Crab,PSA Initial_to_peak Crab,PSA Peak_to_recovery Crab,PSA Average Crab
0,N2,0.110784,0.107783,0.253141,0.217929,0.196599,0.199662,0.178795,0.385499,0.966016,...,40.013364,40.013364,30.362809,30.362809,0.020196,0.013648,0.028023,0.024929,0.017029,0.017439
1,kvs-4,0.128165,0.149204,0.266046,0.226986,0.225901,0.224840,0.450169,0.589937,0.969239,...,37.419393,37.419393,30.105937,30.105937,0.019314,0.016647,0.026998,0.023156,0.017894,0.017940
2,kvs-4,0.106420,0.090803,0.251729,0.212343,0.211656,0.210582,0.333587,0.431421,0.993570,...,38.192508,33.526433,32.088451,27.422376,0.017397,0.014301,0.022863,0.020130,0.014447,0.014542
3,kvs-4,0.131319,0.101250,0.272393,0.241902,0.236100,0.237210,0.438248,0.377352,0.992674,...,35.877767,35.877767,28.511097,28.511097,0.019656,0.015173,0.027831,0.023743,0.019165,0.019181
4,kvs-5,0.102820,0.112525,0.220910,0.191743,0.183870,0.183945,0.211150,0.385448,0.959018,...,36.341616,36.341616,27.783108,27.783108,0.015166,0.014887,0.023164,0.019165,0.014536,0.014557


#### 3.1.6 `baseline` = "psa" , `allele` = True

In [25]:
psa_data_allele_byplate = get_output_byplate(psa_data, baseline="psa", allele=True)

print(f"Shape: {psa_data_allele_byplate.shape}")

psa_data_allele_byplate.head()

Shape: (269, 43)


,dataset,PSA Initial Instantaneous Speed,PSA Recovery Instantaneous Speed,PSA Peak Instantaneous Speed,PSA Initial_to_peak Instantaneous Speed,PSA Peak_to_recovery Instantaneous Speed,PSA Average Instantaneous Speed,PSA Initial Bias,PSA Recovery Bias,PSA Peak Bias,...,PSA Peak Curve,PSA Initial_to_peak Curve,PSA Peak_to_recovery Curve,PSA Average Curve,PSA Initial Crab,PSA Recovery Crab,PSA Peak Crab,PSA Initial_to_peak Crab,PSA Peak_to_recovery Crab,PSA Average Crab
0,N2,0.110784,0.107783,0.253141,0.217929,0.196599,0.199662,0.178795,0.385499,0.966016,...,40.013364,40.013364,30.362809,30.362809,0.020196,0.013648,0.028023,0.024929,0.017029,0.017439
1,kvs-4_sy1622,0.128165,0.149204,0.266046,0.226986,0.225901,0.224840,0.450169,0.589937,0.969239,...,37.419393,37.419393,30.105937,30.105937,0.019314,0.016647,0.026998,0.023156,0.017894,0.017940
2,kvs-4_tm10514,0.106420,0.090803,0.251729,0.212343,0.211656,0.210582,0.333587,0.431421,0.993570,...,38.192508,33.526433,32.088451,27.422376,0.017397,0.014301,0.022863,0.020130,0.014447,0.014542
3,kvs-4_tm14987,0.131319,0.101250,0.272393,0.241902,0.236100,0.237210,0.438248,0.377352,0.992674,...,35.877767,35.877767,28.511097,28.511097,0.019656,0.015173,0.027831,0.023743,0.019165,0.019181
4,kvs-5_tm6152,0.102820,0.112525,0.220910,0.191743,0.183870,0.183945,0.211150,0.385448,0.959018,...,36.341616,36.341616,27.783108,27.783108,0.015166,0.014887,0.023164,0.019165,0.014536,0.014557


## 3.2 Calculate Mean Distances and CIs

In [26]:

def extract_phenotypes(df):
    ''' 
    Splits a multi-column DataFrame into a list of DataFrames, each containing one phenotype

    input: 
        df (pd.DataFrame): dataframe with multiple columns (1st column is the index, the other are phenotypes)

    returns:
        list_phenotypes_df: list with 2 columns - one for index and one for phenotype, 
            for how many phenotypes there are in the input
    '''
    list_phenotypes_df = []
    index = df.columns[0]
    for i in df.columns[1:]:
        list_phenotypes_df.append(df[[index, i]].copy())

    return list_phenotypes_df



def ci95(df):
    """
    input: df of 4 columns: index, mean, count, std

    returns: df of 6 columns: index, mean, count, std, ci95_hi, ci95_low

    """
    for metric in df.columns.levels[0]:
        if metric == 'Gene':
            pass
        else:
            ci95_hi = []
            ci95_lo = []
            for i in df[metric].index:
                m = df[metric]['mean'].loc[i]
                c = df[metric]['count'].loc[i]
                s = df[metric]['sem'].loc[i]
                ci95_hi.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[1])
                ci95_lo.append(stats.t.interval(confidence=0.95, df=c-1, loc=m, scale=s)[0])
            df[metric,'ci95_hi'] = ci95_hi
            df[metric,'ci95_lo'] = ci95_lo
            # df[metric,'ci95']=list(zip(ci95_lo,ci95_hi))
            
    return df



def calculate_MSD(list_of_dfs, by):
    new_list_of_dfs = []
    
    for df in list_of_dfs:
        # Get phenotype column name (assuming 2nd column is the metric)
        pheno_col = df.columns[1]
        
        # Calculate statistics
        stats = df.groupby(by)[df.columns[1]].agg(['mean', 'count', 'sem'])

        
        # Convert to MultiIndex if needed (more robust version)
        if not isinstance(stats.columns, pd.MultiIndex):
            stats.columns = pd.MultiIndex.from_tuples([(pheno_col, col) for col in stats.columns])
        
        # Calculate CI
        stats_2 = ci95(stats)
        
        # Get N2 control data
        if Screen == "Neuron_Genes_Screen":
            N2_mask = stats_2.index == 'N2' if by == "Gene" else stats_2.index.isin(['N2_XJ1','N2_N2'])
        else:
            N2_mask = stats_2.index == 'N2'
            
        N2_data = stats_2[N2_mask]
        
        # Subtract N2 values
        stats_2.iloc[:, 0] -= N2_data.iloc[0, 0]  # mean
        stats_2.iloc[:, 3] -= N2_data.iloc[0, 0]  # ci95_hi
        stats_2.iloc[:, 4] -= N2_data.iloc[0, 0]  # ci95_low
        
        new_list_of_dfs.append(stats_2)
    
    return new_list_of_dfs

In [27]:
def calculate_MSD(list_of_dfs, by):
    new_list_of_dfs = []
    
    for df in list_of_dfs:
        # Get phenotype column name (assuming 2nd column is the metric)
        pheno_col = df.columns[1]
        
        # Create proper MultiIndex structure
        stats = df.groupby(by)[df.columns[1]].agg(['mean', 'count', 'sem'])

        # Convert to MultiIndex if needed (more robust version)
        if not isinstance(stats.columns, pd.MultiIndex):
            stats.columns = pd.MultiIndex.from_tuples([(pheno_col, col) for col in stats.columns])
        
        # Calculate CIs
        stats_2 = ci95(stats)
        
        # Get N2 control data
        if Screen == "Neuron_Genes_Screen":
            N2_mask = stats_2.index == 'N2' if by == "Gene" else stats_2.index.isin(['N2_XJ1','N2_N2'])
        else:
            N2_mask = stats_2.index == 'N2'
            
        N2_data = stats_2[N2_mask]
        
        # Subtract N2 values
        stats_2.iloc[:, 0] -= N2_data.iloc[0, 0]  # mean
        stats_2.iloc[:, 3] -= N2_data.iloc[0, 0]  # ci95_hi
        stats_2.iloc[:, 4] -= N2_data.iloc[0, 0]  # ci95_low
        
        new_list_of_dfs.append(stats_2)
    
    return new_list_of_dfs

In [28]:
def get_MSD(list_MSD):
    '''
    input: List of dataframes, each representing a phenotype with calculated MSD.

    returns: Single combined dataframe joining all input dataframes with MSD values.
    '''
    for a in list_MSD:
        if a.columns.levels[0] == list_MSD[0].columns.levels[0]:
            MSD=a
        else:
            MSD=MSD.join(a)
    return MSD

In [29]:
def get_combined_MSD(baseline_byplate,tap_byplate, psa_byplate, by=['Gene','dataset']):
    """
    Combines MSD datafram from baseline plates and tap plates

    input:
        - baseline_byplate: baseline data by plate
        - tap_byplate: tap data by plate
        - by: what to group by "Gene" or "dataset"
    returns:
        - combined MSD dataframe
    """
    list_baseline_MSD=calculate_MSD(extract_phenotypes(baseline_byplate), by=by)

    list_tap_MSD=calculate_MSD(extract_phenotypes(tap_byplate), by=by)

    list_psa_MSD=calculate_MSD(extract_phenotypes(psa_byplate), by=by)

    baseline_MSD = get_MSD(list_baseline_MSD)
    
    tap_MSD = get_MSD(list_tap_MSD)

    psa_MSD = get_MSD(list_psa_MSD)

    combined_MSD = pd.merge(pd.merge(baseline_MSD, tap_MSD, on=by, how='outer'), psa_MSD, on=by, how='outer')

    combined_MSD=combined_MSD.rename(columns={"habit_dura":"Habituation of Response Duration",
                                         "habit_prob": "Habituation of Respones Probability",
                                         "habit_speed":"Habituation of Response Speed",
                                         "init_dura": "Initial Response Duration",
                                         "init_prob": "Initial Response Probability",
                                         "init_speed": "Initial Response Speed",
                                         "final_dura": "Final Response Duration",
                                         "final_prob": "Final Response Probability",
                                         "final_speed": "Final Response Speed",
                                         "recovery_dura": "Spontaneous Recovery of Response Duration",
                                         "recovery_prob": "Spontaneous Recovery of Response Probability",
                                         "recovery_speed": "Spontaneous Recovery of Response Speed",
                                         "memory_retention_dura": "Memory Retention of Response Duration",
                                         "memory_retention_prob": "Memory Retention of Response Probability",
                                         "memory_retention_speed": "Memory Retention of Response Speed"})

    combined_MSD=combined_MSD.reset_index()
    combined_MSD.columns = combined_MSD.columns.to_flat_index().str.join('-')
    combined_MSD=combined_MSD.rename(columns={by+"-": by})
    combined_MSD['Screen']=Screen
    
    return combined_MSD

### 3.2.1 Gene-level SMD

In [30]:
combined_MSD=get_combined_MSD(baseline_output_byplate,
                              tap_data_byplate, 
                              psa_data_byplate,
                              by='Gene')

combined_MSD.head()

,Gene,Instantaneous Speed-mean,Instantaneous Speed-count,Instantaneous Speed-sem,Instantaneous Speed-ci95_hi,Instantaneous Speed-ci95_lo,Interval Speed-mean,Interval Speed-count,Interval Speed-sem,Interval Speed-ci95_hi,...,PSA Peak_to_recovery Crab-count,PSA Peak_to_recovery Crab-sem,PSA Peak_to_recovery Crab-ci95_hi,PSA Peak_to_recovery Crab-ci95_lo,PSA Average Crab-mean,PSA Average Crab-count,PSA Average Crab-sem,PSA Average Crab-ci95_hi,PSA Average Crab-ci95_lo,Screen
0,F33H2.3,-0.004855,3,0.009873,0.037626,-0.047335,-0.012565,3,0.006896,0.017104,...,3,0.000352,-0.003087,-0.006121,-0.004368,3,0.000535,-0.002065,-0.006671,ASD_WGS_Screen
1,N2,0.000000,63,0.002586,0.005170,-0.005170,0.000000,63,0.002397,0.004792,...,63,0.000320,0.000639,-0.000639,0.000000,63,0.000328,0.000655,-0.000655,ASD_WGS_Screen
2,egl-13,0.045143,3,0.012479,0.098836,-0.008551,0.014590,3,0.007067,0.044998,...,3,0.000318,-0.001816,-0.004551,-0.003228,3,0.000271,-0.002062,-0.004394,ASD_WGS_Screen
3,egl-3,-0.022265,5,0.009416,0.003878,-0.048408,-0.029732,5,0.010916,0.000576,...,5,0.000920,-0.003400,-0.008508,-0.005836,5,0.000904,-0.003327,-0.008344,ASD_WGS_Screen
4,exp-2,0.000144,3,0.002494,0.010876,-0.010589,-0.010518,3,0.002666,0.000955,...,3,0.001354,0.002664,-0.008992,-0.004039,3,0.000507,-0.001858,-0.006221,ASD_WGS_Screen


### 3.2.2 Allele-level SMD

In [31]:
allele_combined_MSD=get_combined_MSD(baseline_output_allele_byplate,
                                     tap_data_allele_byplate, 
                                     psa_data_allele_byplate,
                                     by='dataset')

allele_combined_MSD.head()

,dataset,Instantaneous Speed-mean,Instantaneous Speed-count,Instantaneous Speed-sem,Instantaneous Speed-ci95_hi,Instantaneous Speed-ci95_lo,Interval Speed-mean,Interval Speed-count,Interval Speed-sem,Interval Speed-ci95_hi,...,PSA Peak_to_recovery Crab-count,PSA Peak_to_recovery Crab-sem,PSA Peak_to_recovery Crab-ci95_hi,PSA Peak_to_recovery Crab-ci95_lo,PSA Average Crab-mean,PSA Average Crab-count,PSA Average Crab-sem,PSA Average Crab-ci95_hi,PSA Average Crab-ci95_lo,Screen
0,F33H2.3_tm1628,-0.004855,3,0.009873,0.037626,-0.047335,-0.012565,3,0.006896,0.017104,...,3,0.000352,-0.003087,-0.006121,-0.004368,3,0.000535,-0.002065,-0.006671,ASD_WGS_Screen
1,N2,0.000000,63,0.002586,0.005170,-0.005170,0.000000,63,0.002397,0.004792,...,63,0.000320,0.000639,-0.000639,0.000000,63,0.000328,0.000655,-0.000655,ASD_WGS_Screen
2,egl-13_n483,0.045143,3,0.012479,0.098836,-0.008551,0.014590,3,0.007067,0.044998,...,3,0.000318,-0.001816,-0.004551,-0.003228,3,0.000271,-0.002062,-0.004394,ASD_WGS_Screen
3,egl-3_tm11515,-0.037428,3,0.002062,-0.028557,-0.046298,-0.047495,3,0.000769,-0.044185,...,3,0.000345,-0.005890,-0.008858,-0.007246,3,0.000065,-0.006964,-0.007528,ASD_WGS_Screen
4,egl-3_tm8225,0.000480,2,0.003413,0.043844,-0.042884,-0.003087,2,0.002571,0.029575,...,2,0.000735,0.005519,-0.013166,-0.003720,2,0.000832,0.006846,-0.014287,ASD_WGS_Screen


## 3.3 T-Stat analysis

In [32]:
def baseline_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframe and list of metrics for baseline analysis

    input:
        by (list): what to group by "Gene" or "dataset"
        
    returns:
        list_baseline_Tstats: dataframes to store t-statistics
        list_baseline_metrics: dataframes to store metic names
    """
    PD_baseline_instantspeed_T=pd.DataFrame(columns = [by,"Instantaneous Speed"])
    PD_baseline_intspeed_T=pd.DataFrame(columns = [by,"Interval Speed"])
    PD_baseline_bias_T=pd.DataFrame(columns = [by,"Bias"])
    PD_baseline_morphwidth_T=pd.DataFrame(columns = [by,"Morphwidth"])
    PD_baseline_midline_T=pd.DataFrame(columns = [by,"Midline"])
    PD_baseline_area_T=pd.DataFrame(columns = [by,"Area"])
    PD_baseline_angularspeed_T=pd.DataFrame(columns = [by,"Angular Speed"])
    PD_baseline_aspectratio_T=pd.DataFrame(columns = [by,"Aspect Ratio"])
    PD_baseline_kink_T=pd.DataFrame(columns = [by,"Kink"])
    PD_baseline_curve_T=pd.DataFrame(columns = [by,"Curve"])
    PD_baseline_crab_T=pd.DataFrame(columns = [by,"Crab"])
    PD_baseline_pathlength_T=pd.DataFrame(columns = [by,"Pathlength"])

    list_baseline_Tstats=[PD_baseline_instantspeed_T,
                        PD_baseline_intspeed_T,
                        PD_baseline_bias_T,
                        PD_baseline_morphwidth_T,
                        PD_baseline_midline_T,
                        PD_baseline_area_T,
                        PD_baseline_angularspeed_T,
                        PD_baseline_aspectratio_T,
                        PD_baseline_kink_T,
                        PD_baseline_curve_T,
                        PD_baseline_crab_T,
                        PD_baseline_pathlength_T]

    list_baseline_metrics=["Instantaneous Speed",
                        "Interval Speed",
                        "Bias",
                        "Morphwidth",
                        "Midline",
                        "Area",
                        "Angular Speed",
                        "Aspect Ratio",
                        "Kink",
                        "Curve",
                        "Crab",
                        "Pathlength"]
    
    return list_baseline_Tstats, list_baseline_metrics

In [33]:
def tap_metrics(by=["Gene","dataset"]):
    """
    Create a list of empty dataframes and list of metrics for tap analysis

    input:
        by (list): what to group by "Gene" or "dataset"
        
    returns:
        list_tap_Tstats: dataframes to store t-statistics
        list_tap_metrics: dataframes to store metic names
    """
    recovery_dura=pd.DataFrame(columns = [by,"Recovery Duration"])
    recovery_prob=pd.DataFrame(columns = [by,"Recovery Probability"])
    recovery_speed=pd.DataFrame(columns = [by,"Recovery Speed"])
    memory_retention_dura=pd.DataFrame(columns = [by,"Memory Retention Duration"])
    memory_retention_prob=pd.DataFrame(columns = [by,"Memory Retention Probability"])
    memory_retention_speed=pd.DataFrame(columns = [by,"Memory Retention Speed"])
    init_dura=pd.DataFrame(columns = [by,"Initial Duration"])
    init_prob=pd.DataFrame(columns = [by,"Initial Probability"])
    init_speed=pd.DataFrame(columns = [by,"Initial Speed"])
    final_dura=pd.DataFrame(columns = [by,"Final Duration"])
    final_prob=pd.DataFrame(columns = [by,"Final Probability"])
    final_speed=pd.DataFrame(columns = [by,"Final Speed"])
    hab_dura=pd.DataFrame(columns = [by,"Habituation of Duration"])
    hab_prob=pd.DataFrame(columns = [by,"Habituation of Probability"])
    hab_speed=pd.DataFrame(columns = [by,"Habituation of Speed"])

    list_tap_Tstats = [recovery_dura,
                    recovery_prob,
                    recovery_speed,
                    memory_retention_dura,
                    memory_retention_prob,
                    memory_retention_speed,
                    init_dura,
                    init_prob,
                    init_speed,
                    final_dura,
                    final_prob,
                    final_speed,
                    hab_dura,
                    hab_prob,
                    hab_speed]
    
    list_tap_metrics = ["recovery_dura",
                        "recovery_prob",
                        "recovery_speed",
                        "memory_retention_dura",
                        "memory_retention_prob",
                        "memory_retention_speed",
                        "init_dura",
                        "init_prob",
                        "init_speed",
                        "final_dura",
                        "final_prob",
                        "final_speed",
                        "habit_dura",
                        "habit_prob",
                        "habit_speed"]
    
    return list_tap_Tstats, list_tap_metrics

In [34]:
def psa_metrics(by=["Gene", "dataset"]):
    """
    Create a list of empty dataframes and list of metric names for PSA summary analysis.

    input:
        by (list): what to group by ("Gene" or "dataset")

    returns:
        list_psa_Tstats: list of empty DataFrames for t-statistics
        list_psa_metrics: list of metric names (short strings)
    """

    psa_initial_speed = pd.DataFrame(columns=[by,"PSA Initial Instantaneous Speed"])
    psa_recovery_speed = pd.DataFrame(columns=[by,"PSA Recovery Instantaneous Speed"])
    psa_peak_speed = pd.DataFrame(columns=[by,"PSA Peak Instantaneous Speed"])
    psa_initial_to_peak_speed = pd.DataFrame(columns=[by,"PSA Initial_to_peak Instantaneous Speed"])
    psa_peak_to_recovery_speed = pd.DataFrame(columns=[by,"PSA Peak_to_recovery Instantaneous Speed"])
    psa_avg_speed = pd.DataFrame(columns=[by,"PSA Average Instantaneous Speed"])

    psa_initial_bias = pd.DataFrame(columns=[by,"PSA Initial Bias"])
    psa_recovery_bias = pd.DataFrame(columns=[by,"PSA Recovery Bias"])
    psa_peak_bias = pd.DataFrame(columns=[by,"PSA Peak Bias"])
    psa_initial_to_peak_bias = pd.DataFrame(columns=[by,"PSA Initial_to_peak Bias"])
    psa_peak_to_recovery_bias = pd.DataFrame(columns=[by,"PSA Peak_to_recovery Bias"])
    psa_avg_bias = pd.DataFrame(columns=[by,"PSA Average Bias"])

    psa_initial_ang_speed = pd.DataFrame(columns=[by,"PSA Initial Angular Speed"])
    psa_recovery_ang_speed = pd.DataFrame(columns=[by,"PSA Recovery Angular Speed"])
    psa_peak_ang_speed = pd.DataFrame(columns=[by,"PSA Peak Angular Speed"])
    psa_initial_to_peak_ang_speed = pd.DataFrame(columns=[by,"PSA Initial_to_peak Angular Speed"])
    psa_peak_to_recovery_ang_speed = pd.DataFrame(columns=[by,"PSA Peak_to_recovery Angular Speed"])
    psa_avg_ang_speed = pd.DataFrame(columns=[by,"PSA Average Angular Speed"])

    psa_initial_aspect = pd.DataFrame(columns=[by,"PSA Initial Aspect Ratio"])
    psa_recovery_aspect = pd.DataFrame(columns=[by,"PSA Recovery Aspect Ratio"])
    psa_peak_aspect = pd.DataFrame(columns=[by,"PSA Peak Aspect Ratio"])
    psa_initial_to_peak_aspect = pd.DataFrame(columns=[by,"PSA Initial_to_peak Aspect Ratio"])
    psa_peak_to_recovery_aspect = pd.DataFrame(columns=[by,"PSA Peak_to_recovery Aspect Ratio"])
    psa_avg_aspect = pd.DataFrame(columns=[by,"PSA Average Aspect Ratio"])

    psa_initial_kink = pd.DataFrame(columns=[by,"PSA Initial Kink"])
    psa_recovery_kink = pd.DataFrame(columns=[by,"PSA Recovery Kink"])
    psa_peak_kink = pd.DataFrame(columns=[by,"PSA Peak Kink"])
    psa_initial_to_peak_kink = pd.DataFrame(columns=[by,"PSA Initial_to_peak Kink"])
    psa_peak_to_recovery_kink = pd.DataFrame(columns=[by,"PSA Peak_to_recovery Kink"])
    psa_avg_kink = pd.DataFrame(columns=[by,"PSA Average Kink"])

    psa_initial_curve = pd.DataFrame(columns=[by,"PSA Initial Curve"])
    psa_recovery_curve = pd.DataFrame(columns=[by,"PSA Recovery Curve"])
    psa_peak_curve = pd.DataFrame(columns=[by,"PSA Peak Curve"])
    psa_initial_to_peak_curve = pd.DataFrame(columns=[by,"PSA Initial_to_peak Curve"])
    psa_peak_to_recovery_curve = pd.DataFrame(columns=[by,"PSA Peak_to_recovery Curve"])
    psa_avg_curve = pd.DataFrame(columns=[by,"PSA Average Curve"])

    psa_initial_crab = pd.DataFrame(columns=[by,"PSA Initial Crab"])
    psa_recovery_crab = pd.DataFrame(columns=[by,"PSA Recovery Crab"])
    psa_peak_crab = pd.DataFrame(columns=[by,"PSA Peak Crab"])
    psa_initial_to_peak_crab = pd.DataFrame(columns=[by,"PSA Initial_to_peak Crab"])
    psa_peak_to_recovery_crab = pd.DataFrame(columns=[by,"PSA Peak_to_recovery Crab"])
    psa_avg_crab = pd.DataFrame(columns=[by,"PSA Average Crab"])

    list_psa_Tstats = [
        psa_initial_speed, psa_recovery_speed, psa_peak_speed,
        psa_initial_to_peak_speed, psa_peak_to_recovery_speed, psa_avg_speed,

        psa_initial_bias, psa_recovery_bias, psa_peak_bias,
        psa_initial_to_peak_bias, psa_peak_to_recovery_bias, psa_avg_bias,

        psa_initial_ang_speed, psa_recovery_ang_speed, psa_peak_ang_speed,
        psa_initial_to_peak_ang_speed, psa_peak_to_recovery_ang_speed, psa_avg_ang_speed,

        psa_initial_aspect, psa_recovery_aspect, psa_peak_aspect,
        psa_initial_to_peak_aspect, psa_peak_to_recovery_aspect, psa_avg_aspect,

        psa_initial_kink, psa_recovery_kink, psa_peak_kink,
        psa_initial_to_peak_kink, psa_peak_to_recovery_kink, psa_avg_kink,

        psa_initial_curve, psa_recovery_curve, psa_peak_curve,
        psa_initial_to_peak_curve, psa_peak_to_recovery_curve, psa_avg_curve,

        psa_initial_crab, psa_recovery_crab, psa_peak_crab,
        psa_initial_to_peak_crab, psa_peak_to_recovery_crab, psa_avg_crab
    ]

    list_psa_metrics = [
    "PSA Initial Instantaneous Speed",
    "PSA Recovery Instantaneous Speed",
    "PSA Peak Instantaneous Speed",
    "PSA Initial_to_peak Instantaneous Speed",
    "PSA Peak_to_recovery Instantaneous Speed",
    "PSA Average Instantaneous Speed",

    "PSA Initial Bias",
    "PSA Recovery Bias",
    "PSA Peak Bias",
    "PSA Initial_to_peak Bias",
    "PSA Peak_to_recovery Bias",
    "PSA Average Bias",

    "PSA Initial Angular Speed",
    "PSA Recovery Angular Speed",
    "PSA Peak Angular Speed",
    "PSA Initial_to_peak Angular Speed",
    "PSA Peak_to_recovery Angular Speed",
    "PSA Average Angular Speed",

    "PSA Initial Aspect Ratio",
    "PSA Recovery Aspect Ratio",
    "PSA Peak Aspect Ratio",
    "PSA Initial_to_peak Aspect Ratio",
    "PSA Peak_to_recovery Aspect Ratio",
    "PSA Average Aspect Ratio",

    "PSA Initial Kink",
    "PSA Recovery Kink",
    "PSA Peak Kink",
    "PSA Initial_to_peak Kink",
    "PSA Peak_to_recovery Kink",
    "PSA Average Kink",

    "PSA Initial Curve",
    "PSA Recovery Curve",
    "PSA Peak Curve",
    "PSA Initial_to_peak Curve",
    "PSA Peak_to_recovery Curve",
    "PSA Average Curve",

    "PSA Initial Crab",
    "PSA Recovery Crab",
    "PSA Peak Crab",
    "PSA Initial_to_peak Crab",
    "PSA Peak_to_recovery Crab",
    "PSA Average Crab"
]
    
    return list_psa_Tstats, list_psa_metrics


In [35]:
def TTest(Type, DF_ref, output, by=["Gene", "dataset"]):
    """
    Perform two sample t-test for each unique Gene/dataset column in the Df_ref
    input: 
        - a:column name of values 
        - DF_ref:reference dataframe
        - output: output df to store results in 
        - by: what to group by "Gene" or "dataset"
        
    """
    for a in DF_ref[by].unique():
        Tstat_a = ttest_ind(DF_ref[DF_ref.dataset == a][Type], DF_ref[DF_ref.Allele.isin(["XJ1","N2"])][Type],equal_var=False)[0]
        Tstat_g = ttest_ind(DF_ref[DF_ref.Gene == a][Type], DF_ref[DF_ref.Gene == "N2"][Type],equal_var=False)[0]
        Tstat = Tstat_g if by=="Gene" else Tstat_a
        row = [a, Tstat]
        output.loc[len(output)] = row
    # print(output)

def do_TTest(by=["Gene", "dataset"], baseline=["true", "false", "psa"]):
    """
    Perform TTest function for each unique Gene/dataset column in baseline_output/tap_data
    
    input: 
        - by: what to group by "Gene" or "dataset"
        - baseline: whether or not to use baseline data

    returns: sorted T-statistics dataframe
    """

    if baseline=="true":
        list_Tstats, list_metrics = baseline_metrics(by)
        data = baseline_output
    elif baseline=="false":
        list_Tstats,list_metrics = tap_metrics(by)
        data = tap_data
    else:
        list_Tstats,list_metrics = psa_metrics(by)
        data = psa_data
    for x in data[by].unique():
        if Screen=="Neuron_Genes_Screen":
            condition = x in (["N2"] if by == "Gene" else ["N2_XJ1", "N2_N2"])
        else:
            condition = (x =="N2")
        if condition:
            pass
        else:
            output_gene=data[data[by]==x]
            gene_data=data[data['Date'].isin(output_gene['Date'].unique())]
            if Screen=="Neuron_Genes_Screen":
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])] if by=="Gene" else gene_data[gene_data[by].isin(['N2_N2','N2_XJ1', x])]
            else:
                gene_data_final = gene_data[gene_data[by].isin(['N2', x])]

            for a,b in zip(list_metrics, list_Tstats):
                TTest(a, gene_data_final, b, by) # calls t test function
    
    PD_Tstats=pd.DataFrame()
    for a in list_Tstats:
        b=a.groupby([by], as_index=False).mean()
        if b.columns.values[1] == list_Tstats[0].columns.values[1]:
            PD_Tstats=b
        else:
            PD_Tstats=PD_Tstats.join(b.iloc[:,1])
            
    PD_Tstats=PD_Tstats.set_index(by)
    
    return PD_Tstats
            

### T-stat on Baseline data:

### 3.3.1 Allele-level T-stat analysis of baseline data

In [36]:
warnings.filterwarnings('ignore')

PD_baseline_Tstats_allele = do_TTest("dataset", baseline="true") # get sorted T-statistics DataFrame 

# PD_baseline_Tstats_allele_sorted=PD_baseline_Tstats_allele.sort_index()

PD_baseline_Tstats_allele.head()

,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
dataset,,,,,,,,,,,,
F33H2.3_tm1628,12.680386,-31.834561,38.993540,-101.645543,-68.296568,-132.735371,24.782306,7.825293,1.810935,-63.427507,11.076581,90.317435
N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
egl-13_n483,123.955844,87.581723,61.085181,54.911775,125.436396,236.012683,62.301487,-34.116632,-35.716702,-20.800653,61.253247,76.756323
egl-3_tm11515,-255.919282,-302.174526,-108.794011,-85.978384,-205.381340,-386.515397,10.997738,177.727918,59.386568,171.868590,-38.745748,-130.922618
egl-3_tm8225,23.005338,2.065338,24.657334,-29.653238,-13.584484,-24.835767,6.866533,-51.043968,-35.492816,-55.120200,-0.499409,110.980897


### 3.3.2 Gene-level T-stat analysis of baseline data

In [37]:
warnings.filterwarnings('ignore')

PD_baseline_Tstats=do_TTest("Gene", baseline="true") # get sorted T-statistics DataFrame 

# PD_baseline_Tstats_sorted=PD_baseline_Tstats.sort_index()

PD_baseline_Tstats.head()

,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
Gene,,,,,,,,,,,,
F33H2.3,12.680386,-31.834561,38.993540,-101.645543,-68.296568,-132.735371,24.782306,7.825293,1.810935,-63.427507,11.076581,90.317435
N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
egl-13,123.955844,87.581723,61.085181,54.911775,125.436396,236.012683,62.301487,-34.116632,-35.716702,-20.800653,61.253247,76.756323
egl-3,-118.859645,-145.253995,-64.777924,-76.051618,-143.033970,-183.197584,12.080417,100.083179,36.090340,100.902745,-29.804171,-19.549876
exp-2,30.753838,-36.641847,108.452982,36.213577,-221.424735,-130.986046,39.738167,-26.935328,-86.591230,-218.472246,9.393604,-34.668982


### T-stat analysis for tap-response data:

### 3.3.3 Allele level T-stat analysis of tap response data

In [38]:
warnings.filterwarnings('ignore')

PD_habituation_Tstats_allele = do_TTest("dataset", baseline="false") # get sorted T-statistics DataFrame 

# PD_habituation_Tstats_allele_sorted=PD_habituation_Tstats_allele.sort_index()

PD_habituation_Tstats_allele.head()

,Recovery Duration,Recovery Probability,Recovery Speed,Memory Retention Duration,Memory Retention Probability,Memory Retention Speed,Initial Duration,Initial Probability,Initial Speed,Final Duration,Final Probability,Final Speed,Habituation of Duration,Habituation of Probability,Habituation of Speed
dataset,,,,,,,,,,,,,,,
F33H2.3_tm1628,2.091921,-0.410414,1.403282,3.552380,-2.398374,0.848310,2.597581,0.322818,-0.373236,-1.714904,3.562572,0.148247,4.403699,-3.877424,-0.295273
N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
egl-13_n483,1.465319,0.641831,0.075848,2.085280,1.118264,0.482187,1.257981,0.589023,0.232180,-0.892945,0.400771,-0.422373,2.133465,-0.011149,0.472465
egl-3_tm11515,-0.692825,-1.389914,1.591163,-1.556195,-2.090175,-2.974764,-0.517692,-2.051992,-16.321865,-0.014193,-3.633485,-1.677616,-0.483650,-0.928980,-2.677582
egl-3_tm8225,3.412880,-0.437777,2.054571,2.026106,-1.633451,2.529606,-0.005831,1.721430,-1.132192,-4.366703,4.157431,-0.387798,1.055092,-2.817100,-0.208502


### 3.3.4 Gene-level T-stat analysis of Tap response data

In [ ]:
warnings.filterwarnings('ignore')

PD_habituation_Tstats = do_TTest("Gene", baseline="false") # get sorted T-statistics DataFrame 

PD_habituation_Tstats_sorted=PD_habituation_Tstats.sort_index()

PD_habituation_Tstats.head()

,Recovery Duration,Recovery Probability,Recovery Speed,Memory Retention Duration,Memory Retention Probability,Memory Retention Speed,Initial Duration,Initial Probability,Initial Speed,Final Duration,Final Probability,Final Speed,Habituation of Duration,Habituation of Probability,Habituation of Speed
Gene,,,,,,,,,,,,,,,
F33H2.3,2.091921,-0.410414,1.403282,3.552380,-2.398374,0.848310,2.597581,0.322818,-0.373236,-1.714904,3.562572,0.148247,4.403699,-3.877424,-0.295273
N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
egl-13,1.465319,0.641831,0.075848,2.085280,1.118264,0.482187,1.257981,0.589023,0.232180,-0.892945,0.400771,-0.422373,2.133465,-0.011149,0.472465
egl-3,-0.030998,-1.350846,1.936844,-0.288136,-2.522821,-0.979063,-0.518625,-1.368894,-3.392139,-0.538919,-0.541231,-1.589889,-0.236200,-1.398848,-2.055436
exp-2,1.856516,-0.918935,-1.257068,3.416996,-4.423082,-0.073341,4.912882,3.905008,0.112998,2.644841,7.529645,-1.405021,3.481890,-2.515468,0.994785
flp-20,0.400911,0.396168,0.070782,0.336746,-0.343892,0.408619,0.121590,-1.229302,-0.651514,0.427805,0.238584,-0.871632,-0.034844,-1.088292,0.424443
fubl-1,0.538013,-0.313832,0.363057,-0.924473,0.810713,0.023240,-1.169103,-1.117702,-0.106201,2.113670,-4.550766,0.411622,-1.980642,2.165387,-0.395960
fubl-2,-0.231493,0.222683,1.607090,0.432426,-1.031484,2.773428,0.042221,0.377378,0.813471,-2.174910,1.810949,-0.106157,0.833627,-0.535646,0.465075
gtbp-1,-1.181652,1.119478,-0.341119,2.361534,-2.333230,3.018076,1.808972,1.873705,3.605351,-1.735782,3.185687,-3.076854,3.050452,-3.194833,4.544171


### T-stat analysis for psa data:

### 3.3.5 Allele level T-stat analysis of PSA data

In [40]:
warnings.filterwarnings('ignore')

psa_tstats_allele = do_TTest("dataset", baseline="psa") # get sorted T-statistics DataFrame 

psa_tstats_allele.head()

,PSA Initial Instantaneous Speed,PSA Recovery Instantaneous Speed,PSA Peak Instantaneous Speed,PSA Initial_to_peak Instantaneous Speed,PSA Peak_to_recovery Instantaneous Speed,PSA Average Instantaneous Speed,PSA Initial Bias,PSA Recovery Bias,PSA Peak Bias,PSA Initial_to_peak Bias,...,PSA Peak Curve,PSA Initial_to_peak Curve,PSA Peak_to_recovery Curve,PSA Average Curve,PSA Initial Crab,PSA Recovery Crab,PSA Peak Crab,PSA Initial_to_peak Crab,PSA Peak_to_recovery Crab,PSA Average Crab
dataset,,,,,,,,,,,,,,,,,,,,,
F33H2.3_tm1628,0.856089,-4.862981,-7.521956,-4.269037,-5.237061,-4.801596,10.925248,11.648821,-1.419482,5.324057,...,5.873338,-15.596996,19.263740,-10.949385,-12.192963,7.852230,-19.274981,-18.590091,-11.787066,-7.328706
N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
egl-13_n483,6.577766,9.514866,0.780058,0.542613,-3.119057,-2.487719,-3.053471,-4.470684,15.798427,-1.883245,...,23.276308,-14.714290,27.915063,14.877784,3.297685,22.921792,-9.772569,-1.911962,1.928236,1.789996
egl-3_tm11515,-73.734870,-23.731393,-167.873039,-141.789579,-95.191774,-120.086457,-49.910566,-28.269271,-16.572248,-23.660354,...,20.555044,6.051965,36.011684,71.090862,-56.903790,-12.600325,-51.598078,-119.682383,-64.388484,-85.552311
egl-3_tm8225,-0.286540,1.250603,-0.754965,-2.212848,0.346396,0.066762,2.361477,13.931365,14.283740,2.989756,...,3.658082,3.916833,0.397699,0.037464,23.626446,2.653692,-12.178441,1.843506,-5.064457,-3.305653


### 3.3.6 Gene-level T-stat analysis of PSA data

In [41]:
warnings.filterwarnings('ignore')

psa_tstats = do_TTest("Gene", baseline="psa") # get sorted T-statistics DataFrame 

psa_tstats.head()

,PSA Initial Instantaneous Speed,PSA Recovery Instantaneous Speed,PSA Peak Instantaneous Speed,PSA Initial_to_peak Instantaneous Speed,PSA Peak_to_recovery Instantaneous Speed,PSA Average Instantaneous Speed,PSA Initial Bias,PSA Recovery Bias,PSA Peak Bias,PSA Initial_to_peak Bias,...,PSA Peak Curve,PSA Initial_to_peak Curve,PSA Peak_to_recovery Curve,PSA Average Curve,PSA Initial Crab,PSA Recovery Crab,PSA Peak Crab,PSA Initial_to_peak Crab,PSA Peak_to_recovery Crab,PSA Average Crab
Gene,,,,,,,,,,,,,,,,,,,,,
F33H2.3,0.856089,-4.862981,-7.521956,-4.269037,-5.237061,-4.801596,10.925248,11.648821,-1.419482,5.324057,...,5.873338,-15.596996,19.263740,-10.949385,-12.192963,7.852230,-19.274981,-18.590091,-11.787066,-7.328706
N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
egl-13,6.577766,9.514866,0.780058,0.542613,-3.119057,-2.487719,-3.053471,-4.470684,15.798427,-1.883245,...,23.276308,-14.714290,27.915063,14.877784,3.297685,22.921792,-9.772569,-1.911962,1.928236,1.789996
egl-3,-17.946823,-13.348460,-19.021139,-19.423705,-18.043373,-18.369289,-14.041270,-5.683513,-11.072495,-13.310380,...,14.928615,6.711533,16.804447,18.175966,-12.857733,-8.312603,-22.522894,-17.917255,-20.955246,-20.225394
exp-2,1.551406,-1.437682,-35.691528,-13.416911,-15.116727,-13.706659,22.255844,24.411979,20.513424,22.386943,...,-3.893347,-3.355829,-14.865384,-15.211090,10.932612,11.731963,-17.720373,-5.894425,0.805955,-8.675914


# 4. Merging t-stat data into one dataset

In [42]:
def pop_cols(combined):
    """
    Reorders columns in the combined dataframe. 
    (pops specific columns["Area", "Midline", "Morphwidth", "Angular Speed"] and
    reinserts at different positions)

    input:
        combined: dataframe with columns to be reordered

    returns: 
        NA    
        
    """
    first_col=combined.pop("Area")
    combined.insert(0,"Area",first_col)

    first_col=combined.pop("Midline")
    combined.insert(0,"Midline",first_col)

    first_col=combined.pop("Morphwidth")
    combined.insert(0,"Morphwidth",first_col)

    first_col=combined.pop("Angular Speed")
    combined.insert(5,"Angular Speed",first_col)

def pop_last(combined):
    """
    Reorders the last three columns of the combined dataframe.
    input:
        combined: dataframe with columns to be reordered

    """
    last_col=combined.pop("Spontaneous Recovery of Response Duration")
    combined.insert(26,"Spontaneous Recovery of Response Duration",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Probability")
    combined.insert(26,"Spontaneous Recovery of Response Probability",last_col)

    last_col=combined.pop("Spontaneous Recovery of Response Speed")
    combined.insert(26,"Spontaneous Recovery of Response Speed",last_col)

    last_col=combined.pop("Memory Retention of Response Duration")
    combined.insert(26,"Memory Retention of Response Duration",last_col)

    last_col=combined.pop("Memory Retention of Response Probability")
    combined.insert(26,"Memory Retention of Response Probability",last_col)

    last_col=combined.pop("Memory Retention of Response Speed")
    combined.insert(26,"Memory Retention of Response Speed",last_col)

def rename_columns(df):
    '''
    Renames columns in the input dataframe
    input:
        combined: dataframe with columns to be renamed   
    returns:
        input dataframe with renamed columns 
    '''
    renames = {
        "Habituation of Duration": "Habituation of Response Duration",
        "Habituation of Probability": "Habituation of Respones Probability",
        "Habituation of Speed": "Habituation of Response Speed",
        "Initial Duration": "Initial Response Duration",
        "Initial Probability": "Initial Response Probability",
        "Initial Speed": "Initial Response Speed",
        "Final Duration": "Final Response Duration",
        "Final Probability": "Final Response Probability",
        "Final Speed": "Final Response Speed",
        "Recovery Duration": "Spontaneous Recovery of Response Duration",
        "Recovery Probability": "Spontaneous Recovery of Response Probability",
        "Recovery Speed": "Spontaneous Recovery of Response Speed",
        "Memory Retention Duration": "Memory Retention of Response Duration",
        "Memory Retention Probability": "Memory Retention of Response Probability",
        "Memory Retention Speed": "Memory Retention of Response Speed"
    }
    return df.rename(columns=renames)

def merge_Tstats(baseline, habituation, by=["Gene", "dataset"], Screen=Screen, psa=False):
    """
    merge baseline and tap response dataframes based on the Gene/dataset
    normalize the merged dataframe and then return it with melted version

    input:
        - baseline: baseline dataframe to merge
        - habituation: habituation dataframe to merge
        - by: what to group by "Gene" or "dataset"
    """

    #merge baseline and habituation data
    combined_Tstats = pd.merge(baseline, habituation, on=by, how='left')
    combined_Tstats = combined_Tstats.sort_index() # sort by index

    # ------------ NORMALISATION STEPS TO BE MOVED TO DASHBOARD -------------------
    # # normalise combined dataframe by subtracting mean and div by sd
    # combined_Tstats_normalized = (combined_Tstats-combined_Tstats.mean())/combined_Tstats.std()

    # if by=="dataset" and Screen=="Neuron_Genes_Screen":
    #     combined_Tstats_normalized_2 = combined_Tstats-combined_Tstats[combined_Tstats.index=="N2_XJ1"].squeeze()
    # else :
    #     combined_Tstats_normalized_2 = combined_Tstats-combined_Tstats[combined_Tstats.index=="N2"].squeeze()  

    pop_cols(combined_Tstats) # reorder columns

    # Skip this step if data = psa
    if not psa:
        #rename columns of combined and normalized df
        combined_Tstats = rename_columns(combined_Tstats)
        # combined_Tstats_normalized_2=rename_columns(combined_Tstats_normalized_2)
        pop_cols(combined_Tstats) # reorder columns
        pop_last(combined_Tstats) # reorder columns

    # -------------- PIVOTING STEPS TO BE MOVED TO DASHBOARD ---------------------
    # # Melt the combined dataframe
    # combined_Tstats_melted=combined_Tstats.reset_index()
    # combined_Tstats_melted=pd.melt(combined_Tstats_melted, id_vars=[by],
    #                             var_name='Metric',
    #                             value_name='T_score')
    
    # # Sort the melted dataframe by T_score
    # combined_Tstats_melted_sorted=combined_Tstats_melted.sort_values(by=['T_score'])

    # # Melt the normalized dataframe
    # combined_Tstats_normalized_melted=combined_Tstats.reset_index()
    # combined_Tstats_normalized_melted=pd.melt(combined_Tstats_normalized_melted, id_vars=[by],
    #                                                var_name='Metric',
    #                                                value_name='T_score')

    # add Screen column to df and its melted version
    combined_Tstats['Screen']=Screen
    # combined_Tstats_normalized_melted['Screen']=Screen

    return combined_Tstats#, combined_Tstats_normalized_melted



## 4.1 Gene-level

- Pass Tap and baseline through merge_Tstats() as df1
- Pass PSA and baseline through merge_Tstats()as df2
- pd.merge df1 and df2 using all columns of baseline

In [43]:
# Baseline + Tap
combined_Tstats = merge_Tstats(PD_baseline_Tstats, PD_habituation_Tstats, "Gene")

In [44]:
# Baseline + PSA 
combined_Tstats_psa = merge_Tstats(
    PD_baseline_Tstats, psa_tstats, by="Gene", psa=True
)

In [45]:
# Baseline + Tap + PSA
final_tstat = pd.merge(combined_Tstats.reset_index(), combined_Tstats_psa.reset_index(), on = PD_baseline_Tstats.columns.to_list().append(['Gene','Screen']), how = 'inner')

final_tstat.head()

,Gene,Morphwidth,Midline,Area,Instantaneous Speed,Interval Speed,Angular Speed,Bias,Aspect Ratio,Kink,...,PSA Peak Curve,PSA Initial_to_peak Curve,PSA Peak_to_recovery Curve,PSA Average Curve,PSA Initial Crab,PSA Recovery Crab,PSA Peak Crab,PSA Initial_to_peak Crab,PSA Peak_to_recovery Crab,PSA Average Crab
0,F33H2.3,-101.645543,-68.296568,-132.735371,12.680386,-31.834561,24.782306,38.993540,7.825293,1.810935,...,5.873338,-15.596996,19.263740,-10.949385,-12.192963,7.852230,-19.274981,-18.590091,-11.787066,-7.328706
1,N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,egl-13,54.911775,125.436396,236.012683,123.955844,87.581723,62.301487,61.085181,-34.116632,-35.716702,...,23.276308,-14.714290,27.915063,14.877784,3.297685,22.921792,-9.772569,-1.911962,1.928236,1.789996
3,egl-3,-76.051618,-143.033970,-183.197584,-118.859645,-145.253995,12.080417,-64.777924,100.083179,36.090340,...,14.928615,6.711533,16.804447,18.175966,-12.857733,-8.312603,-22.522894,-17.917255,-20.955246,-20.225394
4,exp-2,36.213577,-221.424735,-130.986046,30.753838,-36.641847,39.738167,108.452982,-26.935328,-86.591230,...,-3.893347,-3.355829,-14.865384,-15.211090,10.932612,11.731963,-17.720373,-5.894425,0.805955,-8.675914


In [47]:
# # Baseline + Tap + PSA melted
# final_tstat_melted = pd.concat([combined_Tstats_normalized_melted, combined_Tstats_psa_melted]).drop_duplicates()

# final_tstat_melted.head()

## 4.2 Allele level 


- Pass Tap and baseline through merge_Tstats() as df3
- Pass PSA and baseline through merge_Tstats()as df4
- pd.merge df3 and df4 using all columns of basline

In [48]:
# Baseline + Tap
combined_Tstats_allele = merge_Tstats(PD_baseline_Tstats_allele,PD_habituation_Tstats_allele, "dataset")

In [49]:
# Baseline + PSA 
combined_Tstats_psa_allele = merge_Tstats(
    PD_baseline_Tstats_allele, psa_tstats_allele, by="dataset", psa=True
)

In [50]:
# Baseline + Tap + PSA
final_tstat_allele = pd.merge(combined_Tstats_allele.reset_index(), combined_Tstats_psa_allele.reset_index(), on = PD_baseline_Tstats_allele.columns.to_list().append(['dataset','Screen']), how = 'outer')

final_tstat_allele.head()

,dataset,Morphwidth,Midline,Area,Instantaneous Speed,Interval Speed,Angular Speed,Bias,Aspect Ratio,Kink,...,PSA Peak Curve,PSA Initial_to_peak Curve,PSA Peak_to_recovery Curve,PSA Average Curve,PSA Initial Crab,PSA Recovery Crab,PSA Peak Crab,PSA Initial_to_peak Crab,PSA Peak_to_recovery Crab,PSA Average Crab
0,F33H2.3_tm1628,-101.645543,-68.296568,-132.735371,12.680386,-31.834561,24.782306,38.993540,7.825293,1.810935,...,5.873338,-15.596996,19.263740,-10.949385,-12.192963,7.852230,-19.274981,-18.590091,-11.787066,-7.328706
1,N2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,egl-13_n483,54.911775,125.436396,236.012683,123.955844,87.581723,62.301487,61.085181,-34.116632,-35.716702,...,23.276308,-14.714290,27.915063,14.877784,3.297685,22.921792,-9.772569,-1.911962,1.928236,1.789996
3,egl-3_tm11515,-85.978384,-205.381340,-386.515397,-255.919282,-302.174526,10.997738,-108.794011,177.727918,59.386568,...,20.555044,6.051965,36.011684,71.090862,-56.903790,-12.600325,-51.598078,-119.682383,-64.388484,-85.552311
4,egl-3_tm8225,-29.653238,-13.584484,-24.835767,23.005338,2.065338,6.866533,24.657334,-51.043968,-35.492816,...,3.658082,3.916833,0.397699,0.037464,23.626446,2.653692,-12.178441,1.843506,-5.064457,-3.305653


In [ ]:
# # Baseline + Tap + PSA melted
# final_tstat_melted_allele = pd.concat([combined_Tstats_normalized_melted_allele, combined_Tstats_psa_melted_allele]).drop_duplicates()

# final_tstat_melted_allele.head()

# 5. Save data to database (sqlite3)

#### A janky way to add data and update the sql 

1. Read table to pd.DataFrame
2. Add new data to pd.DataFrame
3. Replace old table with newly updated pd.DataFrame

# Primary Keys For Each SQL Table:

####  -- Gene_Allele_WormBaseID:
WBGene, WBAllele
#### -- alleleMSD:
dataset, Screen
#### -- gene_MSD:
Gene, Screen
#### -- allele_profile_data:
dataset, Metric, Screen
#### -- gene_profile_data:
Gene, Metric, Screen
#### -- tap_baseline_data:
Time, Plate_id, Date, Screen, dataset
#### -- tap_response_data:
plate, Date, Plate_id, Screen, taps, dataset, Gene, Allele
#### -- tstat_allele_data:
dataset, Screen
#### -- tstat_gene_data:
Gene, Screen
#### -- psa_summarized_data:
Plate_id,Date,Scree,dataset,Gene,Allele

In [ ]:
print(tap_output.head(5))
print(baseline_output.head(5))

tap_output.Screen = Screen
baseline_output.Screen = Screen

print(tap_output.head(5))
print(baseline_output.head(5))

In [51]:

### This code will connect to PostgreSQL database and write non-duplicate data into the database tables.

# Loads database config values from database.ini file and validates that user and password are set.
config = load_config()
if (config['user'] == "" or config['password'] == ""):
    print("Please set your user and password in the database.ini file.")
    sys.exit(1)
    
# Creates a connection pool to PostgreSQL database using SQLAlchemy.
engine = create_engine(f"postgresql+psycopg://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

# Function to insert data into PostgreSQL table, skipping duplicates based on primary keys.
def postgres_skip_on_duplicate(pd_table, conn, keys, data_iter):
    data = [dict(zip(keys,row)) for row in data_iter]
    conn.execute(insert(pd_table.table).on_conflict_do_nothing(), data)

# --------- Write the dataframes to PostgreSQL tables -----------

# Complete tap response data
print("working on tap_output:") 
tap_output.to_sql('tap_response_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# Complete baseline data
print("working on tap_baseline_data:") 
baseline_output.to_sql('tap_baseline_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# Baseline + Tap + PSA combined tstat data by Gene
print("working on tstat_gene_data")
final_tstat.reset_index().to_sql('tstat_gene_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# Baseline + Tap + PSA combined tstat data by Allele
print("working on tstat_allele_data")
final_tstat_allele.reset_index().to_sql('tstat_allele_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# MSD Baseline + Tap + PSA by Gene
print("working on gene_MSD")
combined_MSD.to_sql('gene_MSD', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# MSD Baseline + Tap + PSA by Allele
print("working on allele_MSD")
allele_combined_MSD.to_sql('allele_MSD', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# Summarised PSA data (speed, kink, curve, etc.)
print("working on psa_data:") 
psa_data.to_sql('psa_summarised_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# # Melted Baseline + Tap + PSA combined tstat data by Gene
# print("working on gene_profile_data")
# final_tstat_melted.to_sql('gene_profile_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

# # Melted Baseline + Tap + PSA combined tstat data by Allele
# print("working on allele_profile_data")
# final_tstat_melted_allele.to_sql('allele_profile_data', engine, if_exists='append', index=False, method=postgres_skip_on_duplicate)

working on tap_output:
working on tap_baseline_data:
working on tstat_gene_data
working on tstat_allele_data
working on gene_MSD
working on allele_MSD
working on psa_data:


In [ ]:
# # USE THIS CELL TO UPDATE ALL THE NEED TALBES (Also have baseline_output on the second line)

# conn=sqlite3.connect('/Users/lavanya/Desktop/Lavanya_Test/data_updated2.db')

# tap_output.to_sql('tap_response_data', conn, if_exists='append', index=False)

# baseline_output.to_sql('tap_baseline_data', conn, if_exists='append', index=False)

# combined_Tstats_normalize_2.reset_index().to_sql('tstat_gene_data', conn, if_exists='append', index=False)

# combined_Tstats_normalize_allele_2.reset_index().to_sql('tstat_allele_data', conn, if_exists='append', index=False)

# combined_Tstats_normalized_melted.to_sql('gene_profile_data', conn, if_exists='append', index=False)

# combined_Tstats_normalized_melted_allele.to_sql('allele_profile_data', conn, if_exists='append', index=False)

# combined_MSD.to_sql('gene_MSD', conn, if_exists='append', index=False)

# allele_combined_MSD.to_sql('allele_MSD', conn, if_exists='append', index=False)

# # combined_Tstats_melted_sorted.to_sql('allele_phenotype_data', conn, if_exists='replace', index=False)

# print(conn.total_changes)

# conn.close()


# # Want to test edge cases of pd.to_sql functionality#############